In [ ]:
# SKIP THIS, we don't need to run this again,I already download the data csv from FRED
# Data Acquisition — FRED API only
# Period: from 2017-01-01 (per project spec 2026feb18)
'''
import requests
import pandas as pd

FRED_KEY = "06a54d33db1ebd673697229aa3d99248"
START = "2017-01-01"

def fred_series_to_csv(series_id: str, out_csv: str):
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": FRED_KEY,
        "file_type": "json",
        "observation_start": START,
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    obs = r.json()["observations"]

    df = pd.DataFrame({
        "date": [o["date"] for o in obs],
        series_id: [None if o["value"] == "." else float(o["value"]) for o in obs]
    })
    df.to_csv(out_csv, index=False)
    return df

# Interest Rates, Inflation, Unemployment
fred_series_to_csv("UNRATE", "unrate_raw.csv")         # Unemployment (monthly)
fred_series_to_csv("CPIAUCSL", "cpiaucsl_raw.csv")     # CPI / Inflation (monthly)
fred_series_to_csv("DGS2", "dgs2_raw.csv")             # 2Y Treasury yield (daily)
fred_series_to_csv("DGS5", "dgs5_raw.csv")             # 5Y Treasury yield (daily)
fred_series_to_csv("DGS10", "dgs10_raw.csv")           # 10Y Treasury yield (daily)
'''

In [ ]:
# Yahoo Finance: SPX, USDJPY, GOLD (Jan 01 2017 — Feb 18 2026) ALL Daily
import yfinance as yf
import pandas as pd

START = "2017-01-01"
END = "2026-02-18"

tickers = {
    "^GSPC": "spx_raw.csv",      # S&P 500
    "USDJPY=X": "usdjpy_raw.csv", # US Dollar / Japanese Yen
    "GC=F": "gold_raw.csv",      # Gold futures
}

for ticker, out_csv in tickers.items():
    data = yf.download(ticker, start=START, end=END, progress=False, auto_adjust=True)
    if data.empty:
        print(f"No data for {ticker}")
        continue
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    df = data[["Close"]].reset_index()
    df.columns = ["date", ticker]
    df["date"] = df["date"].dt.strftime("%Y-%m-%d")
    df.to_csv(out_csv, index=False)
    print(f"Saved {out_csv}: {len(df)} rows")

Saved spx_raw.csv: 2293 rows
Saved usdjpy_raw.csv: 2374 rows
Saved gold_raw.csv: 2294 rows
